In [1]:
# 6.00 Problem Set 5
# RSS Feed Filter

import feedparser
import string
import time
from project_util import translate_html
from news_gui import Popup

#-----------------------------------------------------------------------
#
# Problem Set 5

#======================
# Code for retrieving and parsing
# Google and Yahoo News feeds
# Do not change this code
#======================

def process(url):
    """
    Fetches news items from the rss url and parses them.
    Returns a list of NewsStory-s.
    """
    feed = feedparser.parse(url)
    entries = feed.entries
    ret = []
    for entry in entries:
        guid = entry.guid
        title = translate_html(entry.title)
        link = entry.link
        summary = translate_html(entry.summary)
        try:
            subject = translate_html(entry.tags[0]['term'])
        except AttributeError:
            subject = ""
        newsStory = NewsStory(guid, title, subject, summary, link)
        ret.append(newsStory)
    return ret

In [2]:
#======================
# Part 1
# Data structure design
#======================

# Problem 1

# TODO: NewsStory. #DONE
class NewsStory(object):
    def __init__ (self, guid, title, subject, summary, link):
        self.guid = guid
        self.title = title
        self.subject = subject
        self.summary = summary
        self.link = link
    def get_guid(self):
        return self.guid
    def get_title(self):
        return self.title
    def get_subject(self):
        return self.subject
    def get_summary(self):
        return self.summary
    def get_link(self):
        return self.link      

In [3]:
text = '''"Soft's" the new! PINK'''
print type(text)
print(string.punctuation)
print(string.uppercase)
string.uppercase.index('C')

<type 'str'>
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
ABCDEFGHIJKLMNOPQRSTUVWXYZ


2

In [4]:
text = '''"Soft's" the new! PINK'''

def split_text(text):
    assert type(text) == str
    pre_split_text = text
    for character in text:
        if character in string.punctuation:
#             character.replace(character, ' ')
            
#             pre_split_text = text.replace(character, ' ')
#             text[text.index(character)].replace(character,)
            pre_split_text = pre_split_text.replace(character, ' ')
    for character in text:
        if character in string.uppercase:
            pre_split_text = pre_split_text.replace(character, string.lowercase[string.uppercase.index(character)])
    return pre_split_text.split()

print(split_text(text))  

def lower_word(word):
    assert type(word) == str
    lower_word = word
    for character in word:
        if character in string.uppercase:
            lower_word = lower_word.replace(character, string.lowercase[string.uppercase.index(character)])
    return lower_word
print(lower_word('INevitable'))

['soft', 's', 'the', 'new', 'pink']
inevitable


In [5]:
#======================
# Part 2
# Triggers
#======================

class Trigger:
    def evaluate(self, story):
        """
        Returns True if an alert should be generated
        for the given news item, or False otherwise.
        """
        raise NotImplementedError

# Whole Word Triggers
# Problems 2-5


# TODO: WordTrigger: DONE

class WordTrigger(Trigger):
    def __init__(self, word):
        self.word = lower_word(word)
    def is_word_in(self, text):
        if self.word in split_text(text):
            return True
        else: 
            return False

# TODO: TitleTrigger: DONE

class TitleTrigger(WordTrigger):
    def __init__(self, word):
        WordTrigger.__init__(self, word)
    def evaluate(self, story):
        title = story.get_title()
        if self.is_word_in(title):
            return True
        else:
            return False
        
# TODO: SubjectTrigger: DONE

class SubjectTrigger(WordTrigger):
    def __init__(self, word):
        WordTrigger.__init__(self, word)
    def evaluate(self, story):
        subject = story.get_subject()
        if self.is_word_in(subject):
            return True
        else: 
            return False
        
# TODO: SummaryTrigger: DONE

class SummaryTrigger(WordTrigger):
    def __init__(self, word):
        WordTrigger.__init__(self, word)
    def evaluate(self, story):
        summary = story.get_summary()
        if self.is_word_in(summary):
            return True
        else:
            return False

In [6]:
# Composite Triggers
# Problems 6-8

# TODO: NotTrigger: DONE

class NotTrigger(Trigger):
    def __init__(self, antitrigger):
        self.antitrigger = antitrigger
    def evaluate(self, story):
        if not self.antitrigger.evaluate(story):
            return True
        else:
            return False
        
# TODO: AndTrigger:DONE 

class AndTrigger(Trigger):
    def __init__(self, trigger1, trigger2):
        self.trigger1 = trigger1
        self.trigger2 = trigger2
    def evaluate(self, story):
        if self.trigger1.evaluate(story) and self.trigger2.evaluate(story):
            return True
        else:
            return False
        
# TODO: OrTrigger: DONE

class OrTrigger(Trigger):
    def __init__(self, trigger1, trigger2):
        self.trigger1 = trigger1
        self.trigger2 = trigger2
    def evaluate(self, story):
        if self.trigger1.evaluate(story) or self.trigger2.evaluate(story):
            return True
        else:
            return False

In [7]:
# Phrase Trigger
# Question 9

# TODO: PhraseTrigger: DONE

class PhraseTrigger(Trigger):
    def __init__(self, phrase):
        assert type(phrase) == str
        self.phrase = phrase
    def evaluate(self, story):
        if self.phrase in (story.get_subject() or story.get_title() or story.get_summary()):
            return True
        else: 
            return False 

In [8]:
#======================
# Part 3
# Filtering
#======================

def filter_stories(stories, triggerlist):
    """
    Takes in a list of NewsStory-s.
    Returns only those stories for whom
    a trigger in triggerlist fires.
    """
    # TODO: Problem 10
    # This is a placeholder (we're just returning all the stories, with no filtering) 
    # Feel free to change this line!
    filtered_stories = []
    for story in stories:
        for trigger in triggerlist:
            if trigger.evaluate(story):
                filtered_stories.append(story)
                break
    return filtered_stories

In [9]:
# test1 = "t1 TITLE world"
# split_test = test1.split()
# dict_trig = {}
# dict_trig[trigger] = split_test[0]
trigger ='t1'    
exec("%s = %ls" % (trigger,[1,2,3]))
print t1

[1, 2, 3]


In [10]:
#======================
# Part 4
# User-Specified Triggers
#======================

def readTriggerConfig(filename):
    """
    Returns a list of trigger objects
    that correspond to the rules set
    in the file filename
    """
    # Here's some code that we give you
    # to read in the file and eliminate
    # blank lines and comments
    triggerfile = open(filename, "r")
    all = [ line.rstrip() for line in triggerfile.readlines() ]
    lines = []
    for line in all:
        if len(line) == 0 or line[0] == '#':
            continue
        lines.append(line)

    # TODO: Problem 11
    # 'lines' has a list of lines you need to parse
    # Build a set of triggers from it and
    # return the appropriate ones
    triggers = []
    trigger_dict = {}
    for line in lines:
        if line.split()[0] != 'ADD':
            if line.split()[1] == 'TITLE':
                trigger = TitleTrigger(line.split()[2]) 
            elif line.split()[1] == 'SUBJECT':
                trigger = SubjectTrigger(line.split()[2]) 
            elif line.split()[1] == 'SUMMARY':
                trigger = SummaryTrigger(line.split()[2]) 
            elif line.split()[1] == 'NOT':
                trigger = NotTrigger(line.split()[2]) 
            elif line.split()[1] == 'AND':
                trigger = AndTrigger(trigger_dict[line.split()[2]], trigger_dict[line.split()[3]]) 
            elif line.split()[1] == 'OR':
                trigger = OrTrigger(trigger_dict[line.split()[2]], trigger_dict[line.split()[3]]) 
            elif line.split()[1] == 'PHRASE':
                trigger = PhraseTrigger(" ".join(line.split()[2:])) 
            else:
                return None

            trigger_dict[line.split()[0]] = trigger

        else:
            for item in line.split()[1:]:
                triggers.append(trigger_dict[item])

    return triggers

In [11]:
triggerlist = readTriggerConfig("triggers.txt")
print triggerlist    

[<__main__.TitleTrigger instance at 0x00000000049C1508>, <__main__.AndTrigger instance at 0x00000000049C14C8>]


In [ ]:
import thread

def main_thread(p):
    # A sample trigger list - you'll replace
    # this with something more configurable in Problem 11
    t1 = SubjectTrigger("Obama")
    t2 = SummaryTrigger("MIT")
    t3 = PhraseTrigger("Supreme Court")
    t4 = OrTrigger(t2, t3)
#     triggerlist = [t1, t4]
    
    # TODO: Problem 11
    # After implementing readTriggerConfig, uncomment this line 
    triggerlist = readTriggerConfig("triggers.txt")

    guidShown = []
    
    while True:
        print "Polling..."

        # Get stories from Google's Top Stories RSS news feed
        stories = process("http://news.google.com/?output=rss")
        # Get stories from Yahoo's Top Stories RSS news feed
        stories.extend(process("http://rss.news.yahoo.com/rss/topstories"))

        # Only select stories we're interested in
        stories = filter_stories(stories, triggerlist)
    
        # Don't print a story if we have already printed it before
        newstories = []
        for story in stories:
            if story.get_guid() not in guidShown:
                newstories.append(story)
        
        for story in newstories:
            guidShown.append(story.get_guid())
            p.newWindow(story)

        print "Sleeping..."
        time.sleep(SLEEPTIME)

SLEEPTIME = 60 #seconds -- how often we poll
if __name__ == '__main__':
    p = Popup()
    thread.start_new_thread(main_thread, (p,))
    p.start()



Polling...
Sleeping...
